<a href="https://colab.research.google.com/github/namenoyo/SQL-Gen-script/blob/main/Master_sql_query_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import HTML

HTML('''

<!DOCTYPE html>
<html lang="th">
<head>
  <meta charset="UTF-8">
  <title>SQL Query Generator</title>
  <style>
    body { font-family: sans-serif; padding: 20px; background: #fff3e0; }
    label { display: inline-block; width: 180px; margin-bottom: 5px; }
    input { margin-bottom: 10px; }
    textarea { width: 100%; height: 300px; margin-top: 20px; font-family: monospace; }
    .section { margin-bottom: 20px; }
    button { background-color: orange; color: black; padding: 6px 14px; margin-right: 10px; border: none; cursor: pointer; }
    button:hover { background-color: darkorange; }
  </style>
</head>
<body>
  <h2>SQL Query Generator</h2>
  <p>สำหรับเตรียมหาข้อมูล ของกร สามัญ เพื่อไป Gen งวดต่อใน AS400</p>

  <div class="section">
    <label>ต้องการใช้วันที่ปัจจุบัน</label>
    <button onclick="useToday()">Gen Date today</button><br>
    <label>ต้องการระบุวันที่เอง</label>
    <input type="date" id="customDate"> <small>format DD/MM/YYYY ปี พศ</small>
  </div>

  <div><strong>วันที่ระบุ:</strong> <span id="displayDate"></span></div>

  <textarea id="sqlOutput" readonly></textarea>

  <div class="section">
    <button onclick="copySQL()">COPY</button>
    <button onclick="clearSQL()">CLEAR</button>
  </div>

  <script>
    function pad(n) {
      return n.toString().padStart(2, '0');
    }

    function toYYMMDD(date) {
      const d = new Date(date);
      const year = d.getFullYear() - 543;
      const yy = (year ).toString().padStart(2, '0');
      const mm = pad(d.getMonth() + 1);
      const dd = pad(d.getDate());
      return { yy, mm, dd };
    }

    function formatThaiDate(date) {
      const d = new Date(date);
      const year = d.getFullYear() + 543;
      return `${year}/${pad(d.getMonth() + 1)}/${pad(d.getDate())}`;
    }

    function generateSQL(dateInput) {
      const base = new Date(dateInput);
      const baseYear = base.getFullYear()+543;
      const baseMonth = base.getMonth() + 1;
      const baseDay = base.getDate();
      const baseYY = (baseYear ).toString().padStart(2, '0');
      //const end = `${(baseYear )}${pad(baseMonth)}${pad(baseDay)}`; //`${baseYY}${pad(baseMonth)}${pad(baseDay)}`;

 const end = `${String(baseYear - 2500).padStart(2, '0')}${pad(baseMonth)}${pad(baseDay)}`;

      const mod1Start = new Date(baseYear, baseMonth - 2, baseDay - 1);
      const mod1End = new Date(baseYear, baseMonth - 1, baseDay - 1);
      const mod1StartStr = `${(mod1Start.getFullYear() - 2500)}${pad(mod1Start.getMonth() + 1)}${pad(mod1Start.getDate())}`;
      const mod1EndStr = `${(mod1End.getFullYear() - 2500)}${pad(mod1End.getMonth() + 1)}${pad(mod1End.getDate())}`;


	const mod3StartDate = new Date(base.getFullYear(), base.getMonth() - 3, 1);
	const mod3EndDate = new Date(base.getFullYear(), base.getMonth() - 3, baseDay);
	const mod3Start = `${String(mod3StartDate.getFullYear()  +543  - 2500).padStart(2, '0')}${pad(mod3StartDate.getMonth() + 1)}01`;
	const mod3End = `${String(mod3EndDate.getFullYear()  +543  - 2500).padStart(2, '0')}${pad(mod3EndDate.getMonth() + 1)}${pad(mod3EndDate.getDate())}`;


    const mod6StartDate = new Date(base.getFullYear(), base.getMonth() - 6, 1);
	const mod6EndDate = new Date(base.getFullYear(), base.getMonth() - 6, baseDay);
	const mod6Start = `${String(mod6StartDate.getFullYear() +543 - 2500 ).padStart(2, '0')}${pad(mod6StartDate.getMonth() + 1)}01`;
	const mod6End = `${String(mod6EndDate.getFullYear() +543  - 2500).padStart(2, '0')}${pad(mod6EndDate.getMonth() + 1)}${pad(mod6EndDate.getDate())}`;


	const mod12StartDate = new Date(base.getFullYear(), base.getMonth() - 12, 1);
	const mod12EndDate = new Date(base.getFullYear(), base.getMonth() - 12, baseDay);
	const mod12Start = `${String(mod12StartDate.getFullYear() +543  - 2500).padStart(2, '0')}${pad(mod12StartDate.getMonth() + 1)}01`;
	const mod12End = `${String(mod12EndDate.getFullYear() +543  - 2500).padStart(2, '0')}${pad(mod12EndDate.getMonth() + 1)}${pad(mod12EndDate.getDate())}`;



      const sql = `SELECT a.POLIC#, a.POSMAS, a.POCMDT, a.POFRDT,a.POTODT, a.POMOD#, a.*
FROM olis.OLPPOLMS AS a
WHERE a.POSTS@ = 'I' AND NOT EXISTS (
  SELECT 1 FROM olis.OLPLOAHD b WHERE a."POLIC#" = b."LHPOL#"
) AND (
  (a.POMOD# = '1' AND a.POTODT BETWEEN '${mod1StartStr}' AND '${mod1EndStr}') OR
  (a.POMOD# = '3' AND a.POFRDT BETWEEN '${mod3Start}' AND '${mod3End}' AND a.POTODT <= '${end}') OR
  (a.POMOD# = '6' AND a.POFRDT BETWEEN '${mod6Start}' AND '${mod6End}' AND a.POTODT <= '${end}') OR
  (a.POMOD# = '12' AND a.POFRDT BETWEEN '${mod12Start}' AND '${mod12End}' AND a.POTODT <= '${end}')
);`;

      document.getElementById('sqlOutput').value = sql;
      document.getElementById('displayDate').textContent = formatThaiDate(dateInput);
    }

    function useToday() {
      const today = new Date();
      const dateStr = today.toISOString().split('T')[0];
      document.getElementById('customDate').value = dateStr;
      generateSQL(dateStr);
    }

    function copySQL() {
      const textarea = document.getElementById('sqlOutput');
      textarea.select();
      document.execCommand('copy');
    }

    function clearSQL() {
      document.getElementById('sqlOutput').value = '';
      document.getElementById('displayDate').textContent = '';
    }

    document.getElementById('customDate').addEventListener('change', function () {
      generateSQL(this.value);
    });
  </script>
</body>
</html>

''')
